In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/AIZOOTech/FaceMaskDetection.git

Cloning into 'FaceMaskDetection'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 198 (delta 11), reused 15 (delta 1), pack-reused 166
Receiving objects: 100% (198/198), 57.77 MiB | 25.72 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F 
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim as optim
import cv2
from tqdm import tqdm
from skimage import io

sys.path.append('/content/FaceMaskDetection/models')

In [ ]:
class MaskDataset(Dataset):
    def __init__(self, img_root_dir, xml_root_dir, transform = None):
      self.ids = [x[:-4] for x in os.listdir(img_root_dir)]
      self.labels = []
      for xml_file in os.listdir(xml_root_dir):
        label = ET.parse(os.path.join(xml_root_dir , xml_file)).getroot().find('object').find('name').text
        if label == 'with_mask':
          self.labels.append(1)
        else:
          self.labels.append(0)
      self.img_root_dir = img_root_dir
      self.xml_root_dir = xml_root_dir
      self.transform = transform
        
    def __len__(self):
        return len(self.ids)
    
    def __getitem__(self, index):
        img_path = os.path.join(r"/content/drive/Shareddrives/deepmask/DeepMask/FaceMaskDataset/archive_2/images", self.ids[index] + '.png')
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        height, width, _ = img.shape
        image_resized = cv2.resize(img, (360,360))
        image_np = image_resized / 255.0  # 归一化到0~1
        image_exp = np.expand_dims(image_np, axis=0)
        image_transposed = image_exp.transpose((0, 3, 1, 2))
        input_img = torch.tensor(image_transposed).float()
        #skimage.color.convert_colorspace()
        y_label = torch.tensor(float(self.labels[index]))
        return (input_img, y_label)

In [ ]:
class Net(nn.Module):
  def __init__(self, pre_model):
    super().__init__()
    self.pre_model = pre_model
  def forward(self, x):
    x = x.squeeze()
    x = self.pre_model(x)
    #print(x.shape())
    x = F.sigmoid(nn.Linear(self.pre_model.fc.in_features, 2)(x))
    return x


In [ ]:
def load_data():
  dataset = MaskDataset(img_root_dir = '/content/drive/Shareddrives/deepmask/DeepMask/FaceMaskDataset/archive_2/images',
                        xml_root_dir = '/content/drive/Shareddrives/deepmask/DeepMask/FaceMaskDataset/archive_2/annotations',
                        transform = None)#transforms.Compose([])#transforms.ToPILImage(),
                                                        #transforms.Resize((256, 256)),
                                                        #transforms.Grayscale(),
                                                        #transforms.ToTensor()]))
                        
  
  train_size = int(0.8 * len(dataset))
  test_size = len(dataset) - train_size
  
  train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

  train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

  return train_loader, test_loader

In [ ]:
def train(model, train_loader, optimizer, epoch):
  model.train()
  correct = 0
  for batch_idx, (data, taget) in tqdm(enumerate(train_loader)):
    optimizer.zero_grad()
    output = model(data)
    loss = F.cross_entropy(output, target)
    loss.backward()
    optimizer.step()

    if batch_idx % 100 == 0:
      print('Train Epoch: {}\\tLoss: {:.6f}'.format(epoch, loss.item()))
  accuracy = test(model, train_loader)
  return accuracy

In [ ]:
def test(model, test_loader):
  model.eval()
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = model(data)
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  accuracy = 100. * correct / len(test_loader.dataset)

  return accuracy

In [ ]:
def main():
  
  #num_epochs = 3
  #learning_rate = 0.001
  #train_loader, test_loader = load_data()
  #model_path = '/content/FaceMaskDetection/models/model360.pth'
  #model = torch.load(model_path)
  #net = Net(model)

  #optimizer = optim.SGD(net.parameters(), lr=learning_rate)
  #train_acc_arr, test_acc_arr = [], []

  #for epoch in range(num_epochs):
  #  train_acc = train(net, train_loader, optimizer, epoch)
  ##  print('\nTrain set Accuracy: {:.0f}%\n'.format(train_acc))
  #  test_acc = test(net, test_loader)
  #  print('\nTrain set Accuracy: {:.0f}%\n'.format(test_acc))
  #  train_acc_arr.append(train_acc)
  #  test_acc_arr.append(test_acc)
  #torch.save(net.state.dict(), "deepMaskModel.pt")


In [ ]:
if __name__ == '__main__':
  main()

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'MainModel.KitModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Mod

In [ ]:
import torch
import sys
import cv2
import numpy as np
import copy
import os
sys.path.append(r'C:\Users\shrey\hackUmass\FaceMaskDetection\models')

model_path = '/content/FaceMaskDetection/models/model360.pth'

model = torch.load(model_path)
model.eval()

for i in os.listdir(r"/content/drive/Shareddrives/deepmask/DeepMask/FaceMaskDataset/archive_2/images"):
    img_path = os.path.join(r"/content/drive/Shareddrives/deepmask/DeepMask/FaceMaskDataset/archive_2/images", i)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width, _ = img.shape
    image_resized = cv2.resize(img, (360,360))
    image_np = image_resized / 255.0  # 归一化到0~1
    image_exp = np.expand_dims(image_np, axis=0)
    image_transposed = image_exp.transpose((0, 3, 1, 2))
    input_img = torch.tensor(image_transposed).float()
    print(input_img.shape)
    a,b = model.forward(input_img)
    a = a.detach()
    b = b.detach()

